In [1]:
import xarray as xr
from dask.distributed import Client

In [2]:
mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')
tmask = 1 - mesh.tmask[0]
volume = ((1-tmask) * mesh.e3t_0[0] * mesh.e1t[0] * mesh.e2t[0]).rename({'z':'depth'})

In [3]:
ww_out_p1 = xr.open_dataset('/data/sallen/results/MEOPAR/Scrubberwater/01jan19_nn/SalishSea_1d_20190101_20190630_scrb_T.nc')
ww_out_p2 = xr.open_dataset('/data/sallen/results/MEOPAR/Scrubberwater/01jul19_nn/SalishSea_1d_20190701_20191231_scrb_T.nc')

In [6]:
xr.concat([ww_out_p1, ww_out_p2], dim='time_counter')

: 